In [1]:
# 图像分类的着手途径：
# 1.直接使用卷积神经网络类似LeNet那样的端到端的方法，根据图像的原始像素输入，做简单的
# 数据处理后得到分类结果，算力要求比较大。
# 2.使用人工特征提取函数（滤波器）首先提取对图像输入提取特征，然后使用机器学习的方法，对
# 提取的特征进行分类，其实这个过程已经包含在了卷积神经网络的隐含层中，但卷积神经网络中
# 的滤波器是学习出来的。
# 有时比较干净的数据集和有效的特征比单纯的提高分类器的效率更有效

# 学习特征表示
# 多层神经网络可以学习得到数据的多级表征，并且逐级表示越来越抽象的概念或是模式。
# 神经网络端到端的学习（即特征提取包含在内）的发展受到限制的几个因素：
# 1.数据：ImageNet数据集的产生推动了计算机视觉和机器学习的研究
# 2.硬件：GPU OpenCL和CUDA之类的编程框架

# AlexNet 8层卷积神经网络 与LeNet类似，但是有显著的特征
# 有5层卷积和2层全连接隐层，以及1个全连接输出层

# AlexNet的优势
# 1.卷积层数更深，对特征的抽象能力更强
# 2.采用ReLU激活函数缓解了梯度消失的影响
# 3.使用了Dropout技巧
# 4.使用了大量的图像增广，如翻转，裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合

# 简化版本的AlexNet
import time
import torch
from torch import nn,optim
import torchvision

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv = nn.Sequential(
            # 第一层卷积层
            nn.Conv2d(1,96,11,4),
            nn.ReLU(),
            nn.MaxPool2d(3,2), #kernel_size ,stride
            #第二层卷积层
            nn.Conv2d(96,256,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            # 第三层卷积层
            nn.Conv2d(256,384,3,1,1),
            nn.ReLU(),
            # 第四层卷积层
            nn.Conv2d(384,384,3,1,1),
            nn.ReLU(),
            # 第五层卷积层
            nn.Conv2d(384,256,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(3,2)
        )
        self.fc = nn.Sequential(
            # 第一个隐层
            nn.Linear(256*5*5,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            # 第二个隐层
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            # 最后的全连接输出层
            nn.Linear(4096,10),
        )
        
    def forward(self,img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0],-1))
        return output
    
    
net = AlexNet()
print(net)




AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (

In [2]:
# 读取数据
def load_data_fashion_mnist(batch_size,resize=None,root = '~/Datasets/FashionMNIST'):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root,train = True,download=True,transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root,train = False,download=True,transform=transform)
    train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=4)
    test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=True,num_workers=4)
    return train_iter,test_iter

batch_size = 128
train_iter,test_iter = load_data_fashion_mnist(batch_size,resize=224)



In [ ]:
# 训练
lr,num_epochs = 0.001,5
optimizer = torch.optim.Adam(net.parameters(),lr = lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cpu
